In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
from pyarrow.csv import read_csv
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import sklearn.svm as svm
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from lightgbm import LGBMClassifier

In [95]:
X = read_csv('X_model.csv').to_pandas()
y = read_csv('Y_model.csv').to_pandas()
X_exam = read_csv('X_exam.csv').to_pandas()
X_exam = X_exam.fillna(0)
X.fillna(0, inplace=True)

In [3]:
personal = X.iloc[:,:3]
dall = X.loc[:,'c20220101':'t20220826']
dXc = X.loc[:,'c20220101':'c20220826']
dXs = X.loc[:,'s20220101':'s20220826']
dXt = X.loc[:,'t20220101':'t20220826']

In [98]:
personal_ex = X_exam.iloc[:,:3]
dall_ex = X_exam.loc[:,'c20220101':'t20220826']
dXc_ex = X_exam.loc[:,'c20220101':'c20220826']
dXs_ex = X_exam.loc[:,'s20220101':'s20220826']
dXt_ex = X_exam.loc[:,'t20220101':'t20220826']

In [99]:
dall_ex

,c20220101,c20220102,c20220103,c20220104,c20220105,c20220106,c20220107,c20220108,c20220109,c20220110,...,t20220817,t20220818,t20220819,t20220820,t20220821,t20220822,t20220823,t20220824,t20220825,t20220826
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0
199997,0.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199998,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
X_np = [dall.iloc[i,:] for i in tqdm(range(len(dall)))]

100%|███████████████████████████████████████████████████████████████████████| 800000/800000 [00:55<00:00, 14403.38it/s]


In [100]:
X_np_ex = [dall_ex.iloc[i,:] for i in tqdm(range(len(dall_ex)))]

100%|███████████████████████████████████████████████████████████████████████| 200000/200000 [00:18<00:00, 10632.48it/s]


In [8]:
for i in tqdm(range(len(X_np))):
    X_np[i] = np.array(X_np[i], dtype=np.int32).reshape(3,34,7)

100%|███████████████████████████████████████████████████████████████████████| 800000/800000 [00:09<00:00, 81504.20it/s]


In [101]:
for i in tqdm(range(len(X_np_ex))):
    X_np_ex[i] = np.array(X_np_ex[i], dtype=np.int32).reshape(3,34,7)

100%|███████████████████████████████████████████████████████████████████████| 200000/200000 [00:02<00:00, 77759.29it/s]


In [102]:
zero0 = [0]*200000

In [103]:
personal_ex['zero0'] = zero0
personal_ex['zero1'] = zero0
personal_ex['zero2'] = zero0
personal_ex['zero3'] = zero0

In [104]:
personal_ex = personal_ex[['gender', 'zero0', 'zero1', 'age_code', 'zero2', 'zero3', 'region_code']]

In [105]:
personal_ex

,gender,zero0,zero1,age_code,zero2,zero3,region_code
0,2,0,0,4,0,0,4
1,2,0,0,10,0,0,15
2,2,0,0,4,0,0,2
3,1,0,0,8,0,0,1
4,2,0,0,8,0,0,2
...,...,...,...,...,...,...,...
199995,2,0,0,6,0,0,7
199996,1,0,0,10,0,0,6
199997,1,0,0,11,0,0,1
199998,1,0,0,4,0,0,16


In [106]:
torch.tensor(np.array(list(personal_ex.iloc[0,:])*3).reshape(3,1,7))

tensor([[[2, 0, 0, 4, 0, 0, 4]],

        [[2, 0, 0, 4, 0, 0, 4]],

        [[2, 0, 0, 4, 0, 0, 4]]], dtype=torch.int32)

In [107]:
X_np_ex = torch.Tensor(X_np_ex)

In [17]:
X_per = [np.array(list(personal.iloc[i,:])*3).reshape(3,1,7) for i in tqdm(range(len(X_np)))]

100%|███████████████████████████████████████████████████████████████████████| 800000/800000 [01:07<00:00, 11852.22it/s]


In [108]:
X_per_ex = [np.array(list(personal_ex.iloc[i,:])*3).reshape(3,1,7) for i in tqdm(range(len(X_np_ex)))]

100%|███████████████████████████████████████████████████████████████████████| 200000/200000 [00:19<00:00, 10000.26it/s]


In [18]:
X_per = torch.tensor(X_per)

In [109]:
X_per_ex = torch.tensor(X_per_ex)

In [110]:
X_per_ex.shape

torch.Size([200000, 3, 1, 7])

In [20]:
X_np.shape

torch.Size([800000, 3, 34, 7])

In [111]:
X_np_ex = torch.cat([X_per_ex, X_np_ex], axis=2)

In [22]:
# Import libraries to use for Deep Learning 

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset
from torchvision.io import read_image
from torchsummary import summary
import pandas as pd
from PIL import Image
import os 

import cv2 as cv2

In [115]:
X_train, X_val, y_train, y_val = train_test_split(X_np, y, 
                                                    stratify = y, shuffle = True, 
                                                    random_state = 42, test_size=0.2) 

In [116]:

train_dataset = TensorDataset(torch.Tensor(np.array(X_train)), torch.Tensor(np.array(y_train))) #, torch.Tensor(np.array(y_train))
valid_dataset = TensorDataset(torch.Tensor(np.array(X_val)), torch.Tensor(np.array(y_val))) #, torch.Tensor(np.array(y_val))
test_dataset = TensorDataset(torch.Tensor(np.array(X_np_ex))) #, torch.Tensor(np.array(y_test))

### YOU CAN USE ANY TRANSFORMS YOU WANT. MAKE IT RUNNABLE!

train_loader = DataLoader(train_dataset, batch_size=1000, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=1000, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

### END OF THE CODE.

In [117]:
### PLEASE WRITE YOUR CODE BELOW.

class SimpleCNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=16, kernel_size=3,
                               stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3,
                               stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=2,
                               stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=2,
                               stride=1, padding=1)
        
        self.avg_pool = nn.AvgPool2d(kernel_size=3, stride=2)

        self.bn1 = nn.BatchNorm2d(num_features=16)
        self.bn2 = nn.BatchNorm2d(num_features=32)
        self.bn3 = nn.BatchNorm2d(num_features=64)
        self.bn4 = nn.BatchNorm2d(num_features=128)

        self.dropout = nn.Dropout(p=0.5)

        self.activation = nn.LeakyReLU()

        self.classifier = nn.Linear(in_features=16*4*4, out_features=num_classes)
        self.sigmoid = torch.sigmoid

    def forward(self, x):
        #print(f'Original input dimension in a format of [B, C, H, W] is {list(x.size())}')
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.activation(out)
        out = self.dropout(out)
        out = self.avg_pool(out)
        # Or simply, 
        # out = self.avg_pool(self.dropout(self.activation(self.bn1(self.conv1(x)))))

        #print(f'After first  [Conv -> BatchNorm -> ReLU -> Dropout -> MaxPool] the dimension of tensor is {list(out.size())}')
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.activation(out)
        out = self.dropout(out)
        out = self.avg_pool(out)
        # Or simply,
        # out = self.avg_pool(self.dropout(self.activation(self.bn2(self.conv2(out)))))

        #print(f'After second [Conv -> BatchNorm -> ReLU -> Dropout -> MaxPool] the dimension of tensor is {list(out.size())}')
        #out = self.conv3(out)
        #out = self.bn3(out)
        #out = self.activation(out)
        #out = self.dropout(out)
        #out = self.avg_pool(out)
        #print(f'After thrid [Conv -> BatchNorm -> ReLU -> Dropout -> MaxPool] the dimension of tensor is {list(out.size())}')

        #out = self.conv4(out)
        #out = self.bn4(out)
        #out = self.activation(out)
        #out = self.dropout(out)
        #out = self.avg_pool(out)

        # Or simply,
        # out = self.avg_pool(self.dropout(self.activation(self.bn3(self.conv3(out)))))

        # Need to flatten the output tensor with the shape of [B, C, H, W] -> [B, N]
        # in order to fed into linear classifier
        #print(f'After fourth  [Conv -> BatchNorm -> ReLU -> Dropout -> MaxPool] the dimension of tensor is {list(out.size())}')

        out = out.reshape(out.size(0), -1)
        #print(out.size())
        
        #print(f'Linear layers requires inputs to be [B, N] dimensions')
        out = self.classifier(out)
        output_score = self.sigmoid(out) # (-1)

        return out, output_score
### END OF THE CODE.

In [118]:
model = SimpleCNN(in_channels=3, num_classes=2).cuda()
summary(model, (3, 35, 7), device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 16, 35, 7]             448
       BatchNorm2d-2            [-1, 16, 35, 7]              32
         LeakyReLU-3            [-1, 16, 35, 7]               0
           Dropout-4            [-1, 16, 35, 7]               0
         AvgPool2d-5            [-1, 16, 17, 3]               0
            Conv2d-6            [-1, 32, 17, 3]           4,640
       BatchNorm2d-7            [-1, 32, 17, 3]              64
         LeakyReLU-8            [-1, 32, 17, 3]               0
           Dropout-9            [-1, 32, 17, 3]               0
        AvgPool2d-10             [-1, 32, 8, 1]               0
           Linear-11                    [-1, 2]             514
Total params: 5,698
Trainable params: 5,698
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/back

In [119]:
### PLEASE FILL OUT THE HYPERPARAMETERS
### NOTE THAT YOU SHOULD SET DIFFERENT PARAMETERS FOR DIFFERENT OPTIMIZERS.

lr = 0.001
epochs = 70

## OPTIMIZER HYPERPARAMETERS - PLEASE ADD/REMOVE DEPENDS ON OPTIMIZER.
betas = (0.9, 0.999)
momentum = 0.9

## WHEN USING GPU, PUT `.cuda()` on model and criterion.

model = SimpleCNN(in_channels=3, num_classes=2).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=betas)
criterion = nn.CrossEntropyLoss().cuda()

In [120]:
def train(model, optimizer, criterion, data_loader, epoch):
    model.train()
    total_loss = 0.0
    for idx, batch in enumerate(data_loader):
        img, target = batch[0].cuda(), batch[1].cuda()
        #img = img.float().permute(0,3,1,2)

        ### PLEASE WRITE YOUR CODE BELOW.
        # Initialize the optimizer
        optimizer.zero_grad()

        # Make a prediction
        output, y_pred_proba = model(img)

        # Calculate loss with prediction and target
        target = target.type(torch.LongTensor)
        loss = criterion(output, target.squeeze(dim=-1).cuda())

        # Compute the gradient
        loss.backward()

        # Update Parameters
        optimizer.step()


        ### END OF THE CODE.

        total_loss += loss.item() 

        if idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch + 1, idx * img.size(0), len(data_loader.dataset),
                100. * idx * img.size(0) / len(data_loader.dataset), 
                loss.data))

    return total_loss / len(data_loader)

def validate(model, criterion, data_loader):
    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    val_prepro = []
    val_targetvalues = []

    with torch.no_grad():
        for idx, batch in enumerate(data_loader):
            img, target = batch[0].cuda(), batch[1].cuda()

            ### PLEASE WRITE YOUR CODE BELOW.

            # Make a prediction
            output = model(img)[0]
            y_pred_proba = model(img)[1].cpu()

            # Calculate validation loss (although it is optional)
            target = target.type(torch.LongTensor)
            loss = criterion(output, target.squeeze(dim=-1).cuda())

            # Get the right prediction - make sure naming the prediction as 'predicted' 
            _, predicted = torch.max(output.data, 1)

            ### END OF THE CODE.

            val_loss += loss.item()
            val_acc += (predicted == target.squeeze(dim=-1).cuda()).sum().item()
            val_targetvalues.extend(target.squeeze(dim=-1).cpu())
            val_prepro.extend(y_pred_proba[:,1].cpu())
            #print(roc_auc_score(target.squeeze(dim=-1).cpu(), y_pred_proba[:,1].cpu()))

        total_val_acc = val_acc / len(data_loader.dataset)

        


        print('\nValidation set: Average loss: {:.4f}, AUROC: ({:.4f})\n'.format(
            val_loss / len(data_loader),
            100. * roc_auc_score(target.squeeze(dim=-1).cpu(), y_pred_proba[:,1].cpu())))
    
    return total_val_acc

In [92]:
def test(model, data_loader):
    model.eval()
    test_acc = 0.0
    targetvalues = []
    test_prepro = []

    with torch.no_grad():
        for idx, batch in enumerate(data_loader):
            img, target = batch[0].cuda(), batch[1].cuda()

            ### PLEASE WRITE YOUR CODE BELOW.

            # Make a prediction
            output = model(img)[0]
            y_pred_proba = model(img)[1].cpu()
            

            # Calculate validation loss (although it is optional)
            target = target.type(torch.LongTensor)
            loss = criterion(output, target.squeeze(dim=-1).cuda())

            # Get the right prediction - make sure naming the prediction as 'predicted' 
            _, predicted = torch.max(output.data, 1)
            test_acc += (predicted == target.squeeze(dim=-1).cuda()).sum().item()
            targetvalues.extend(target.squeeze(dim=-1).cpu())
            test_prepro.extend(y_pred_proba[:,1].cpu())

            ### END OF THE CODE.

        print('\n Test set:  AUROC: ({:.4f})\n'.format(100. * roc_auc_score(target.squeeze(dim=-1).cpu(), y_pred_proba[:,1].cpu())))
        
    return test_prepro, targetvalues

In [121]:
def exam(model, data_loader):
    model.eval()
    test_acc = 0.0
    targetvalues = []
    test_prepro = []

    with torch.no_grad():
        for idx, batch in enumerate(data_loader):
            img = batch[0].cuda()

            ### PLEASE WRITE YOUR CODE BELOW.

            # Make a prediction
            output = model(img)[0]
            y_pred_proba = model(img)[1].cpu()
            test_prepro.extend(y_pred_proba[:,1].cpu())
            

        
    return test_prepro

In [122]:
for epoch in tqdm(range(epochs)):

    ### PLEASE WRITE YOUR CODE BELOW.
    
    # Train your model with train dataloader

    train_loss =  train(model, optimizer, criterion, train_loader, epoch)

    # Validate your model with validation dataloader

    validation_accuracy = validate(model, criterion, valid_loader)

    ### END OF THE CODE.

  0%|                                                                                           | 0/70 [00:00<?, ?it/s]

Train Epoch: 1 [0/640000 (0%)]	Loss: 0.919505
Train Epoch: 1 [100000/640000 (16%)]	Loss: 0.217209
Train Epoch: 1 [200000/640000 (31%)]	Loss: 0.217024
Train Epoch: 1 [300000/640000 (47%)]	Loss: 0.197783
Train Epoch: 1 [400000/640000 (62%)]	Loss: 0.213312
Train Epoch: 1 [500000/640000 (78%)]	Loss: 0.213124
Train Epoch: 1 [600000/640000 (94%)]	Loss: 0.194453


  1%|█▏                                                                                 | 1/70 [00:09<10:35,  9.22s/it]


Validation set: Average loss: 0.1983, AUROC: (82.7695)

Train Epoch: 2 [0/640000 (0%)]	Loss: 0.206340
Train Epoch: 2 [100000/640000 (16%)]	Loss: 0.195692
Train Epoch: 2 [200000/640000 (31%)]	Loss: 0.204834
Train Epoch: 2 [300000/640000 (47%)]	Loss: 0.200996
Train Epoch: 2 [400000/640000 (62%)]	Loss: 0.199306
Train Epoch: 2 [500000/640000 (78%)]	Loss: 0.189545
Train Epoch: 2 [600000/640000 (94%)]	Loss: 0.193942


  3%|██▎                                                                                | 2/70 [00:18<10:21,  9.15s/it]

...


Validation set: Average loss: 0.1751, AUROC: (89.2972)

Train Epoch: 69 [0/640000 (0%)]	Loss: 0.168074
Train Epoch: 69 [100000/640000 (16%)]	Loss: 0.202115
Train Epoch: 69 [200000/640000 (31%)]	Loss: 0.160249
Train Epoch: 69 [300000/640000 (47%)]	Loss: 0.178987
Train Epoch: 69 [400000/640000 (62%)]	Loss: 0.170645
Train Epoch: 69 [500000/640000 (78%)]	Loss: 0.149890
Train Epoch: 69 [600000/640000 (94%)]	Loss: 0.185548


 99%|████████████████████████████████████████████████████████████████████████████████▊ | 69/70 [11:30<00:10, 10.30s/it]


Validation set: Average loss: 0.1749, AUROC: (89.2417)

Train Epoch: 70 [0/640000 (0%)]	Loss: 0.172110
Train Epoch: 70 [100000/640000 (16%)]	Loss: 0.166112
Train Epoch: 70 [200000/640000 (31%)]	Loss: 0.158800
Train Epoch: 70 [300000/640000 (47%)]	Loss: 0.165873
Train Epoch: 70 [400000/640000 (62%)]	Loss: 0.181979
Train Epoch: 70 [500000/640000 (78%)]	Loss: 0.168018
Train Epoch: 70 [600000/640000 (94%)]	Loss: 0.170206


100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [11:41<00:00, 10.01s/it]


Validation set: Average loss: 0.1750, AUROC: (88.5179)



In [94]:
test_prepro, target = test(model, test_loader)


 Test set:  AUROC: (88.8667)



In [83]:
y_reset = y_test.reset_index().drop(columns=['index'])

In [84]:
target = pd.DataFrame(target)

In [129]:
pd.DataFrame(exam(model, test_loader), columns=['task1']).to_csv('submission_task1.csv')